# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [1]:
# install tensorflow, smdebug
!pip install -r requirements.txt

     -------------------------------------- 270.1/270.1 KB 4.1 MB/s eta 0:00:00
     -------------------------------------- 266.3/266.3 MB 5.5 MB/s eta 0:00:00
     -------------------------------------- 895.7/895.7 KB 4.1 MB/s eta 0:00:00
  Using cached wrapt-1.14.1-cp310-cp310-win_amd64.whl (35 kB)
     ---------------------------------------- 2.6/2.6 MB 4.8 MB/s eta 0:00:00
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     ---------------------------------------- 23.2/23.2 MB 9.1 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 11.8 MB/s eta 0:00:00
     ------------------------------------- 439.2/439.2 KB 13.8 MB/s eta 0:00:00
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     ---------------------------------------- 3.7/3.7 MB 14.0 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 10.7 MB/s eta 0:00:00
     --------------------------------------- 14.8/14.8 MB 14.6 MB/s eta 0:00:00
  Using cached typing_ext

  error: subprocess-exited-with-error
  
  × Running setup.py install for pyinstrument-cext did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running install
      running build
      running build_ext
      building 'pyinstrument_cext' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> pyinstrument-cext

note: This is an issue with the package mentioned above, not pip.
hint: See above for output from the failure.
You should consider upgrading via the 'C:\Users\BiggMaathy\Documents\Image-Classification-using-AWS-SageMaker\.env\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import matplotlib.pyplot as plt
import numpy as np
import os
from sagemaker.tensorflow import TensorFlow
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuosParameter, HyperparameterTuner

The PlantVillage dataset consists of 54303 images of healthy and unhealthy leaves, divided into 38 categories by species and disease. This dataset data is structure as follows:

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=uint8),
    'image/filename': Text(shape=(), dtype=string),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=38),
})

Where image field contains the Image itself, the image/filename contains the image filename and the label contains 38 different categories.

For more information see: [An open access repository of images on plant health to enable the development of mobile disease diagnostics](https://arxiv.org/abs/1511.08060).

For details about this dataset see: https://www.tensorflow.org/datasets/catalog/plant_village?hl=pt-br

Donwload dataset: [Plant_leaf_diseases_dataset_without_augmentation.zip](https://data.mendeley.com/datasets/tywbtsjrjv/1)

In [ ]:
#TODO: Fetch and upload the data to AWS S3

# Command to download and unzip data
!wget s3://sagemaker-us-east-1-890701353618/plant_leaf_diseases/plant_leaf_diseases_dataset_without_augmentation.zip
!unzip plant_leaf_diseases_dataset_without_augmentation.zip

In [ ]:
session = sagemaker.Session()
role = get_execution_role()
local_data_dir = "/tmp/data"

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [ ]:
# Define a range of hyperparameters for the model training
hyperparameters_range = {
    "learning_rate": ContinuosParameter(1e-4, 1e-3),
    "batch_size": CategoricalParameter([32, 64, 128, 256, 512])
}

In [ ]:
# Create a TensorFlow estimator for training the model
estimator = TensorFlow(
    # Specify the entry point script for the training job
    entry_point= "train_model.py",
    # Assign the AWS IAM role for the training job
    role = role,
    # Set the number of instances for the training job to 1
    instance_count=1,
    # Specify the instance type for the training job
    instance_type="ml.p3.2xlarge",
    # Set the source directory for the training scripts
    source_dir="scripts",
    # Set the version of TensorFlow to use
    framework_version="2.3.1",
    # Set the version of Python to use
    py_version="py37",
    # Set the hyperparameters for the training job
    hyperparameters={
        "batch-size": 512,
        "epochs": 20
    }
)

# Define a objective metric to tune. Note: the entry point script must return the same metric specified here
objective_metric_name = "average test loss"

# Define the objective (Minimize or Maximize)
objective_type = "Minimize"

# Define the metric definitions such as name and a regular expression needed to 
# extract the metric from the CloudWatch logs of the training job
metric_definitions = [{'Name': 'average test loss', 'Regex': 'average test loss: ([0-9\\.]+)'}]

# Define a HyperparameterTuner 
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameters_range,
    metric_definitions,
    max_jobs=4, # Number of training jobs to run
    max_parallel_jobs=2, # Number of training jobs to run in parallel
    objective_type=objective_type
)

In [ ]:
# Set a prefix for the data stored in the S3 bucket
prefix = "plants"

# Get the default S3 bucket for the current AWS session
bucket = session.default_bucket()

# Upload the local data to the S3 bucket with the specified prefix
loc = session.upload_data(path=local_data_dir, bucket=bucket, key_prefix=prefix)

# Define the input channels for the training and testing data
channels = {"training": loc, "testing": loc}

# Fit the tuner to the input data channels
tuner.fit(inputs=channels)

In [ ]:
# Get the best estimators
best_estimator = tuner.best_estimator()

#Get the hyperparameters of the best trained model
hypers = best_estimator.hyperparameters()
batch_size = hypers["batch-size"] # return a string
learning_rate = hypers["learning-rate"] 

# Deploy best model to a endpoint
predictor = tuner.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge")

# Delete endpoint
predictor.delete_endpoint()

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
# Get hyperparameter from best model

In [ ]:
# TODO: Set up debugging and profiling rules and hooks

In [ ]:
# TODO: Create and fit an estimator

estimator = # TODO: Your estimator here

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy() # TODO: Add your deployment configuration like instance type and number of instances

In [ ]:
# TODO: Run an prediction on the endpoint

image = # TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()